<a href="https://colab.research.google.com/github/icarito/UPSK-SQL001-SQLZoo-murder/blob/sql-murder/Sql-Murder-Mystery/SQL_Murder_Mystery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL Murder Mystery!
Adapted By: Martin Arroyo

![Detective making connections between points](https://github.com/freestackinitiative/coop_sql_notebooks/blob/main/assets/sleuth.png?raw=1)

**Credit**

This material was adapted from the [SQL Murder Mystery by Knight Lab](https://mystery.knightlab.com/) under [Creative Commons CC BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/). The SQL Murder Mystery was originally created by [Joon Park](https://twitter.com/joonparkmusic) and [Cathy He](https://twitter.com/Cathy_MeiyingHe) while they were Knight Lab fellows. See the [GitHub repository](https://github.com/NUKnightLab/sql-mysteries) for more information.

## Escenario

¡Se ha cometido un crimen y los detectives necesitan tu ayuda! Te dieron el informe de la escena del crimen, pero de alguna manera lo perdiste. Recuerdas vagamente que el crimen fue un `murder` _(asesinato)_ que ocurrió en algún momento el 15 de Enero de 2018 y que tuvo lugar en `SQL City`. Depende de ti averiguar quién es el asesino usando solo tus habilidades en SQL y tu ingenio. Te proporcionan una conexión a la base de datos del Departamento de Policía, que tiene todas las pistas que necesitarás para atrapar al asesino.

Usa las habilidades que desarrollaste en SQL 101 y 102, junto con cualquier recurso que desees, para resolver el `SQL Murder Mystery`!"

## Conéctate a la base de datos del Departamento de Policía

Para comenzar y ejecutar tus consultas, presiona play en la celda de abajo para conectarte a la base de datos del Departamento de Policía.

Para ejecutar consultas, crea una nueva celda de `Code` y escribe `%%sql` en la parte superior. Luego puedes escribir tus consultas debajo. Ve el ejemplo a continuación:
```python
%%sql

SELECT *
FROM table

```

In [6]:
%%capture
# @title Press Play { display-mode: "form" }
# Install `teachdb` and `coop_grader`
print("Installing `teachdb` and its dependencies...")
!pip install --quiet --upgrade git+https://github.com/freestackinitiative/teachingdb.git git+https://github.com/martinmarroyo/coop_grader.git
print("Successfully installed `teachdb`")
import pandas as pd
from teachdb.teachdb import connect_teachdb
from coop_grader.sql_murder_mystery.check_suspect import check_suspect
# Set configurations for notebook
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 99)
# Load data
con = connect_teachdb(database="sql_murder_mystery")

%sql con

## Descubriendo tablas en la base de datos

Comenzamos nuestra búsqueda para encontrar al asesino explorando la base de datos del Departamento de Policía. Pero aún no has visto la base de datos y no sabes cuáles son las tablas, así que, ¿cómo sabes qué buscar?

Afortunadamente, la mayoría de los sistemas de gestión de bases de datos relacionales tienen esta información almacenada en un lugar donde puedes consultarla. Muy a menudo, se utiliza un esquema especial conocido como [`information_schema`](https://en.wikipedia.org/wiki/Information_schema) para almacenar información sobre las tablas y columnas en tu base de datos (también conocido como metadata). La base de datos del Departamento de Policía tiene un information schema, con la vista `tables` que te muestra qué tablas están disponibles, y la vista `columns` que te muestra todas las columnas de cada tabla y sus tipos de datos.

### Listando todas las tablas en la base de datos de la Policía

#### Revisa la [descripción de `information_schema.tables`](https://duckdb.org/docs/sql/information_schema.html#tables-and-views)

Primero, veremos todas las tablas disponibles para nosotros consultando la vista `information_schema.tables`. Te daremos la primera consulta para empezar, pero de aquí en adelante tendrás que idear las consultas restantes utilizando tu conocimiento de SQL y tu ingenio.

Aquí está la consulta necesaria para mostrarte las tablas en la base de datos del Departamento de Policía. Cópiala/Pégala en la celda de abajo y ejecútala para ver las tablas disponibles para ti:

>```sql
>SELECT *
>FROM information_schema.tables
>```

In [ ]:
%%sql
/*
  Solicitar todas las columnas de la tabla information_schema.tables
*/
SELECT *
FROM information_schema.tables

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action,TABLE_COMMENT
0,memory,main,crime_scene_report,BASE TABLE,None,None,None,None,None,YES,NO,None,None
1,memory,main,drivers_license,BASE TABLE,None,None,None,None,None,YES,NO,None,None
2,memory,main,facebook_event_checkin,BASE TABLE,None,None,None,None,None,YES,NO,None,None
3,memory,main,get_fit_now_check_in,BASE TABLE,None,None,None,None,None,YES,NO,None,None
4,memory,main,get_fit_now_member,BASE TABLE,None,None,None,None,None,YES,NO,None,None
5,memory,main,income,BASE TABLE,None,None,None,None,None,YES,NO,None,None
6,memory,main,interview,BASE TABLE,None,None,None,None,None,YES,NO,None,None
7,memory,main,person,BASE TABLE,None,None,None,None,None,YES,NO,None,None


### Listando todas las tablas y sus columnas en la base de datos de la Policía

**Revisa la [descripción de `information_schema.columns`](https://duckdb.org/docs/sql/information_schema.html#columns)**

¡Genial! Ahora que sabes qué tablas están disponibles, es hora de averiguar las columnas que tiene cada tabla junto con el tipo de datos de cada columna. Escribe una consulta que muestre el nombre de la tabla, el nombre de la columna y el tipo de datos para cada tabla en la base de datos del Departamento de Policía utilizando la vista `information_schema.columns`. Asegúrate de que la salida esté ordenada por nombre de tabla y luego por nombre de columna (ascendente):

*Pistas opcionales - ¡intenta usarlas solo si te quedas **REALMENTE** atascado!*

<details>
<summary>Pista 1</summary>
<p>Estructura tu consulta de la misma manera que lo hicimos en la consulta anterior donde miramos `information_schema.tables`</p>
</details>

<details>
<summary>Pista 2</summary>
<p>Asegúrate de revisar el enlace para la descripción de information_schema.columns. Te dirá los nombres de las columnas que debes usar para la consulta. Recuerda, queremos el nombre de la tabla, el nombre de la columna y el tipo de datos - ¡revisa la descripción para encontrar los nombres apropiados!</p>
</details>

<details>
<summary>Pista 3</summary>
<p>No olvides usar la declaración ORDER BY para ordenar los resultados de tu búsqueda. Estamos buscando ordenar ambas columnas en orden ascendente, lo cual se puede especificar usando la palabra clave ASC, sin embargo, también es el orden de clasificación predeterminado, por lo que ASC no es necesariamente requerido.</p>
</details>

In [ ]:
%%sql
/*
  consulta que muestre el nombre de la tabla, el nombre de la columna y
  el tipo de datos para cada tabla en la base de datos
  del Departamento de Policía utilizando la vista information_schema.columns
*/
FROM
    information_schema.columns
ORDER BY
    table_name ASC,
    column_name ASC;

,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,numeric_precision,numeric_precision_radix,numeric_scale,datetime_precision,interval_type,interval_precision,character_set_catalog,character_set_schema,character_set_name,collation_catalog,collation_schema,collation_name,domain_catalog,domain_schema,domain_name,udt_catalog,udt_schema,udt_name,scope_catalog,scope_schema,scope_name,maximum_cardinality,dtd_identifier,is_self_referencing,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable,COLUMN_COMMENT
0,memory,main,crime_scene_report,city,4,None,YES,VARCHAR,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,None,NaN,None,None,NaN,None
1,memory,main,crime_scene_report,date,1,None,YES,BIGINT,NaN,NaN,64.0,2.0,0.0,NaN,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,None,NaN,None,None,NaN,None
2,memory,main,crime_scene_report,description,3,None,YES,VARCHAR,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,None,NaN,None,None,NaN,None
3,memory,main,crime_scene_report,type,2,None,YES,VARCHAR,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,None,NaN,None,None,NaN,None
4,memory,main,drivers_license,age,2,None,YES,BIGINT,NaN,NaN,64.0,2.0,0.0,NaN,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,None,NaN,None,None,NaN,None
5,memory,main,drivers_license,car_make,8,None,YES,VARCHAR,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,None,NaN,None,None,NaN,None
6,memory,main,drivers_license,car_model,9,None,YES,VARCHAR,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,None,NaN,None,None,NaN,None
7,memory,main,drivers_license,eye_color,4,None,YES,VARCHAR,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,None,NaN,None,None,NaN,None
8,memory,main,drivers_license,gender,6,None,YES,VARCHAR,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,None,NaN,None,None,NaN,None
9,memory,main,drivers_license,hair_color,5,None,YES,VARCHAR,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,None,NaN,None,None,NaN,None


### Entity Relationship Diagram (Optional)

After doing some detective work and finding the tables in the Police Department database, you discover that there is an ERD! This can be really helpful in your quest to find the murderer. If you like, challenge yourself (and your SQL skills) to continue on by just querying the `information_schema` as needed. Otherwise, you can click the dropdown below to reveal the ERD to help you see the tables and relationships in the Police Department database at a glance:

<details>
  <summary>Check the SQL Murder Mystery ERD</summary>
  <img src="https://github.com/freestackinitiative/coop_sql_notebooks/blob/main/assets/murder_mystery_schema.png?raw=1" alt="ERD for SQL Murder Mystery"/>
</details>

### Diagrama de Relaciones de Entidad (Opcional)

Después de hacer un poco de trabajo de detective y encontrar las tablas en la base de datos del Departamento de Policía, descubres que hay un ERD (Diagrama de Relaciones de Entidad). Esto puede ser realmente útil en tu búsqueda para encontrar al asesino. Si lo prefieres, desafíate a ti mismo (y tus habilidades en SQL) a continuar solo consultando el `information_schema` según sea necesario. De lo contrario, puedes hacer clic en el desplegable a continuación para revelar el ERD que te ayudará a ver las tablas y las relaciones en la base de datos del Departamento de Policía de un vistazo:

<details>
  <summary>Revisa el ERD del SQL Murder Mystery</summary>
  <img src="https://github.com/freestackinitiative/coop_sql_notebooks/blob/main/assets/murder_mystery_schema.png?raw=1" alt="ERD for SQL Murder Mystery"/>
</details>

In [7]:
%%sql
/*
  ver informacion de la tabla crime_scene_report
*/
SELECT *
FROM crime_scene_report;


,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC
1,20180115,murder,Life? Dont talk to me about life.,Albany
2,20180115,murder,"Mama, I killed a man, put a gun against his head...",Reno
3,20180215,murder,REDACTED REDACTED REDACTED,SQL City
4,20180215,murder,Someone killed the guard! He took an arrow to the knee!,SQL City
5,20180115,theft,Big Bully stole my lunch money!,Chicago
6,20180115,fraud,"Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do\n eiusmod tempor inc...",Seattle
7,20170712,theft,"A lone hunter stalks the night, firing arrows into the Darkness.\n There is no hidin...",SQL City
8,20170820,arson,"Wield the Hammer of Sol with honor, Titan, it is a thing of\n legend, both past and ...",SQL City
9,20171110,robbery,The Gjallarhorn shoulder-mounted rocket system was forged from\n the armor of Guardi...,SQL City


___

## Resolviendo el Misterio (y verificando si resolviste el caso)

De aquí en adelante, usarás tus habilidades y las pistas en la base de datos del Departamento de Policía para encontrar al asesino. Cuando creas que has encontrado al asesino, vuelve a esta sección y usa la celda de abajo para verificar tu respuesta y ver si encontraste al asesino:

ejemplo:
```python
check_suspect("Adam Levine")
```

In [ ]:
check_suspect("Jeremy Bowers")

Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge,try querying the interview transcript of the murderer to find the real villain behind this crime.If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries.
Use this same `check_suspect` function with your new suspect to check your answer.


True

## Espacio de Trabajo

Usa las celdas de abajo para escribir tus consultas y trabajar en resolver el misterio. Cuando tengas un sospechoso, verifica tu respuesta usando la [celda arriba de esta](#solving-the-mystery-and-checking-if-you-cracked-the-case) y ejecuta su nombre a través de la función `check_suspect`. Si encuentras al asesino, la función te lo dirá.

No olvides usar `%%sql` en la parte superior de las celdas de `Code` que crees para que tus consultas SQL funcionen. ¡Buena suerte, detectives!

## **FILTRAR DATOS DE LA TABLA CRIME_SCENE_REPORT**

Se filtra el contenido de la tabla CRIME_SCENE_REPORT, para obtener solo la informacion de apropiada: murder, fecha del asesinato (15 de Enero de 2018), y el lugar (SQL City).

In [ ]:
%%sql
/*
  Filtrar la info por "murder", el 15 de enero del 2018 en sql city
*/
SELECT *
FROM crime_scene_report
WHERE type = 'murder' AND date = (20180115) AND city = 'SQL City'

,date,type,description,city
0,20180115,murder,Security footage shows that there were 2 witnesses. The first witness lives at the last house o...,SQL City


## **INFORMACIÓN DE LOS TESTIGOS DEL ASESINATO**

Las imágenes de seguridad muestran que hubo 2 testigos. El primer testigo vive en la última casa de la calle "Northwestern Dr". El segundo testigo, llamado Annabel, vive en algún lugar de la calle "Franklin Ave"

In [ ]:
%%sql
/*
  Primer testigo - ultima casa en "Northwestern Dr"
*/
SELECT *
FROM person
WHERE address_street_name	= 'Northwestern Dr'
ORDER BY address_number DESC
LIMIT 1

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


Morty Schapiro	es el primer testigo

In [ ]:
%%sql
/*
    Segundo testigo, Annabel, - en algun lugar de "Franklin Ave"
*/
SELECT *
FROM person
WHERE address_street_name	= 'Franklin Ave'
AND name LIKE 'Annabel%'
/*
'Annabel', '%' comienza con "Annabel" y que luego tenga cualquier combinación de caracteres después.
*/

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


## **ENCONTRAR ENTREVISTAS DE LOS TESTIGOS**

Una vez que obtuvimos los id de los testigos, buscamos sus entrevistas

In [ ]:
%%sql
/*
    Segundo testigo, Annabel, - en algun lugar de "Franklin Ave"
*/
SELECT *
FROM interview
WHERE person_id IN ('16371', '14887');


,person_id,transcript
0,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last we..."
1,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership nu..."


## **ENCONTRAR AL SOSPECHOSO**

De acuerdo con las entrevistas el sospechoso asistió al gimnasio el 09 de enero,es un hombre que tiene membresía Gold en el gimnasio Get Fit Now Gym. Llevaba una bolsa cuyos primeros numeros eran "48Z" y placa del auto "H42W".

Buscamos la placa que tenga H42W en sus numeros

In [ ]:
%%sql

SELECT *
FROM drivers_license
WHERE plate_number LIKE '%H42W%'

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,183779,21,65,blue,blonde,female,H42W0X,Toyota,Prius
1,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
2,664760,21,71,black,black,male,4H42WR,Nissan,Altima


De las 3 coincidencias, verificar la identidad de los dueños de los autos.

In [ ]:
%%sql

SELECT *
FROM person
WHERE license_id IN ('183779','423327','664760')

,id,name,license_id,address_number,address_street_name,ssn
0,51739,Tushar Chandra,664760,312,Phi St,137882671
1,78193,Maxine Whitely,183779,110,Fisk Rd,137882671
2,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


De esta busqueda, logramos obtener los ID de las 3 personas. Comparamos con los ID de los miembros GOLD con member id "48Z".

In [ ]:
%%sql

SELECT *
FROM get_fit_now_member
WHERE membership_status = 'gold' AND id IN ('48Z7A', '48Z55')

,id,person_id,name,membership_start_date,membership_status
0,48Z7A,28819,Joe Germuska,20160305,gold
1,48Z55,67318,Jeremy Bowers,20160101,gold


Obtenemos una coincidencia, Jeremy Bowers con el person_is 67318.

In [ ]:
check_suspect("Jeremy Bowers")

Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge,try querying the interview transcript of the murderer to find the real villain behind this crime.If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries.
Use this same `check_suspect` function with your new suspect to check your answer.


True

Buscamos la entrevista de Jeremy Bowers

In [ ]:
%%sql

SELECT *
FROM interview
WHERE person_id IN ('67318')

,person_id,transcript
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" ..."


Con esto confirmamos que Jeremy Bowers es el *asesino*, pero fue contratado.
Indica lo siguiente: "Fui contratado por una mujer con mucho dinero. No sé su nombre, pero sé que mide alrededor de 5'5" (65") o 5'7" (67"). Tiene cabello rojo y conduce un Tesla Model S. Sé que asistió al concierto de SQL Symphony 3 veces en diciembre de 2017."

Con esto filtramos la informacion en drivers_license de acuerdo a la descripción

In [ ]:
%%sql

SELECT *
FROM drivers_license
WHERE hair_color = 'red' AND car_make = 'Tesla' AND gender = 'female'

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S
1,291182,65,66,blue,red,female,08CM64,Tesla,Model S
2,918773,48,65,black,red,female,917UU3,Tesla,Model S


con el ID obtenido, determinamos el licence_id de cada dueño para ver sus nombres.

In [ ]:
%%sql

SELECT *
FROM person
WHERE license_id IN ('202298', '291182', '918773')

,id,name,license_id,address_number,address_street_name,ssn
0,78881,Red Korb,918773,107,Camerata Dr,961388910
1,99716,Miranda Priestly,202298,1883,Golden Ave,987756388
2,90700,Regina George,291182,332,Maple Ave,337169072


Comparamos el id de las sospechosas, con el id de quien asistio al concierto


In [ ]:
%%sql

SELECT *
FROM facebook_event_checkin
WHERE person_id IN ('78881',	'99716', '90700')

,person_id,event_id,event_name,date
0,99716,1143,SQL Symphony Concert,20171206
1,99716,1143,SQL Symphony Concert,20171212
2,99716,1143,SQL Symphony Concert,20171229


Miranda Priestly con el id 99716, asistio 3 veces al concierto SQL Symphony. Por lo cual es la a asesina.


In [ ]:
check_suspect("Miranda Priestly")

Congrats, you found the brains behind the murder!
Everyone in SQL City hails you as the greatest SQL detective of all time.
Time to break out the champagne!


True